# Demo story

out of all the possible events that can give rise to an infinite number of chains, we are just simulating and demoing just one chain : delay, crew legality, substitution, repositioning, hotel/transport, policy fallback

delay → legality check → substitution → repositioning → hotel/transport → policy fallback

- Event 1: "UA123 delayed 90 min"
- Event 2: "UA123 delay increases to 210 min"
- Event 3: "SFO spare crew found"
- Event 4: "SFO fog — reposition flight canceled"
- Event 5: "DEN crew identified"
- Event 6: "No reposition flight from DEN"
- Event 7: "Hotel at ORD overbooked"
- Event 8: "Alternative hotel secured + notifications sent"

1. The core of the simulation is **time-driven flight status data**.
    
    You will create a mock flight schedule where:
    
- Flights have planned departure and arrival times.
- As the simulation clock advances, some flights develop delays (you modify delay_minutes dynamically as part of the data update).
- These updates happen independently of any direct agent involvement — the agent only sees the changed data when it queries.
1. The agent’s job is to **continuously monitor the flight data**.
    
    At each clock tick:
    
- The agent queries flight status for flights under its watch.
- The agent notices delay or disruption based on the current data state, not because an external event handed it that knowledge.
1. Other data (crew roster, hotel, repositioning options) is mostly static but still queried as needed.
- Crew data defines duty limits, assigned flights, qualifications.
- Hotel and reposition flight data could also change (for realism) but are not monitored continuously; they are checked when the agent’s reasoning requires.
1. The clock powers the data simulation:
- The clock advances simulated time (could be 1 minute per loop or whatever scale works for demo).
- A separate data update function updates flight delay fields as time advances. For example, after 10 ticks, flight UA123’s delay_minutes field is updated to 90. After 20 ticks, it’s 210.
- The agent doesn’t know these thresholds. It just keeps querying flight data and decides when action is needed.
1. There’s no artificial event queue like “fog at SFO” for the agent to process.
- All the agent sees is that repositioning flight from SFO is delayed or unavailable when it queries.
- The cause (fog) is irrelevant to the agent’s reasoning in this case.

# Agent Buckets

1️⃣ **Status Query Agents**

- Scope: Gather and summarize current state.
- Examples: Flight status agent, crew status agent, weather condition agent.
- Purpose: Provide situational awareness, inform other agents' decisions.
- Nature: Passive/reactive, no direct action beyond informing.

2️⃣ **Action Executor Agents**

- Scope: Carry out specific operational tasks or commands.
- Examples: Crew assigner, hotel booker, transport arranger, communicator.
- Purpose: Translate decisions into changes in the world or system state.
- Nature: Active/doers, often invoked by reasoning agents.

3️⃣ **Rule Evaluator Agents**

- Scope: Check compliance with legal, contractual, or operational rules.
- Examples: Duty legality checker, qualification checker, policy retriever.
- Purpose: Gatekeeper for valid actions, ensure plans follow rules.
- Nature: Deciders/validators.

The current categories form the *foundation*. Other domains reflect either meta-control (planner, optimizer) or domain extensions (customer comms, negotiator) as your system expands.

---

### For demo crew rescheduling:

✅ These three agent domains — **status queries, action executors, rule evaluators** — cover the functional scope needed for the core of the problem:

- You need to observe the situation.
- You need to validate legal and operational constraints.
- You need to act (assign, move, rest, inform).

In this domain:

- Most of the work boils down to these roles.
- Additional agents would usually *fit inside* these categories or serve as **orchestrators/planners** that coordinate among them.

---

Other domains are absolutely possible



In [1]:
import time
import os
from queue import Queue
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, Tool, AgentType
from agents import StatusQueryAgents
from mock_data import flight_schedule_df, crew_roster_df, repositioning_flights_df
from prompt import Prompts

load_dotenv()

status_query_agents = StatusQueryAgents(crew_roster_df=crew_roster_df)



def get_llm(service, model, temperature=0):
    assert service in ["gemini", "openai"]
    if service == "gemini":
        api_key = os.getenv("GEMINI_API_KEY")
        return ChatGoogleGenerativeAI(
            google_api_key=api_key,
            model=model, 
            temperature=temperature,
        )
    elif service == "openai":
        api_key = os.getenv("OPENAI_API_KEY")
        return OpenAI(
            openai_api_key=api_key,
            model=model,
            temperature=temperature,
        )


llm = get_llm(service="openai", model="gpt-4o-mini")

agent = initialize_agent(
    status_query_agents.tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

C:\Users\sar31\AppData\Local\Temp\ipykernel_7648\259832376.py:29: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  return OpenAI(
C:\Users\sar31\AppData\Local\Temp\ipykernel_7648\259832376.py:38: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/mig

In [3]:
prompt = Prompts()

for _, row in flight_schedule_df.iterrows():
    current_flight_data = (
        f"Flight ID: {row['flight_id']}\n"
        f"Status: {row['status']}\n"
        f"Delay minutes: {row['delay_minutes']}\n"
        f"Gate: {row['gate']}\n"
        f"Remarks: {row['remarks']}\n"
        f"Aircraft type: {row['aircraft_type']}"
    )
    
    entry_prompt = prompt.entry_prompt(current_flight_data)
    result = agent.run(entry_prompt)
    print(result)

The crew assigned to flight UA123, Captain Jane Doe and First Officer John Roe, will have sufficient rest after their duty ends at 15:21, allowing them to remain legal for future assignments. No escalation is needed.
I recommend escalating the situation as there are no crew members assigned to flight UA456, making it impossible for the flight to operate.
